In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import metrics
import os, sys
from time import time

parentPath = os.path.abspath("..")

if parentPath not in sys.path:
    sys.path.insert(0, parentPath)
    
from utils.experiment import *

# Load data ** Chose your path before get stated **

In [2]:
path = os.getcwd()
rootDir, _ = path.split('ECG-Arritmia-Paper1')
path = 'ECG-Arritmia-Paper1/Database_extracted/Dataset_with_features'
filename = 'MIT-BIH__DS2_5classes__SCM.csv'
dataset = pd.read_csv(os.path.join(rootDir, path, filename));

In [3]:
dataset = dataset.iloc[:,[0, 1, 2, 3, 4, 8]]

In [4]:
dataset.head()

,Var1,Var2,Var3,Var4,Var5,Var9
0,0.796475,0.703297,2.109765,0.374450,0.882230,0
1,0.859659,0.680851,2.344254,0.451078,0.649940,0
2,0.755753,0.688172,2.286413,0.407302,0.819789,0
3,0.800796,0.703297,2.132419,0.373310,0.999879,0
4,0.790644,0.703297,2.213094,0.398221,0.738899,0


In [5]:
dataset.shape

(49668, 6)

# Data preprocessing

In this step we are going standardize our dataset. The pipeline.pkl will be loaded, and it contains the weights of normalization onf DS1 set.

In [6]:
from sklearn.externals import joblib

pipeline_part = joblib.load('./models/pipeline.pkl')

Separete features from labels.

In [7]:
data_std = pipeline_part.transform(dataset.values[:,:-1])
data_label = dataset.values[:,-1]

# Load classifiers

In [8]:
from sklearn.externals import joblib

random_search = joblib.load('./models/classifiers.pkl')

In [9]:
from sklearn.naive_bayes import GaussianNB

bayes_clf = GaussianNB()
bayes_clf.fit(data_std, data_label)

GaussianNB(priors=None)

In [71]:
svm_clf = random_search['SVM-RBF'].best_estimator_
mlp_clf = random_search['MLP'].best_estimator_

classifiers = {'MLP-96': mlp_clf, 'SVM-RBF': svm_clf, 'Naive-Bayes': bayes_clf}

# **TEST** on DS2 set

In [72]:
clf_outputs = {'true': dict((k, {}) for k in classifiers.keys()), 
               'pred': dict((k, {}) for k in classifiers.keys())}
results = dict((k, {}) for k in classifiers.keys())

for clf in classifiers.keys():
    print(clf)
    clf_outputs['true'][clf][0] = data_label
    start_time = time()
    clf_outputs['pred'][clf][0] = classifiers[clf].predict(data_std)
    end_time = (time() - start_time)/data_std.shape[0]
    
    print('Elapsed time: {}'.format(end_time))
    f = open('Results/test_time-{}.txt'.format(clf), 'w')
    f.write('{}'.format(end_time))
    f.close()
    

results = results_clf(5, clf_outputs['true'], clf_outputs['pred'])

MLP-96
Elapsed time: 1.282779128019424e-06
SVM-RBF
Elapsed time: 0.0003055185619602171
Naive-Bayes
Elapsed time: 1.8957608408750117e-07


/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/leandrobmarinho/Documents/ECG-Arritmia-Paper1/Results_Navar/SCM/utils/experiment.py:155: RuntimeWarning: invalid value encountered in double_scalars
  metrics_class[i,3] = (2*metrics_class[i,2]*metrics_class[i,1])/(metrics_class[i,2] + metrics_class[i,1])
/Users/leandrobmarinho/Documents/ECG-Arritmia-Paper1/Results_Navar/SCM/utils/experiment.py:153: RuntimeWarning: invalid value encountered in double_scalars
  metrics_class[i,2] = TP / (TP + FP)


# Save results to CSVs and figures

In [73]:
results

{'MLP-96': {'acc': [0.9085729242167996],
  'average': {'acc': 0.9085729242167996,
   'confMat': array([[4.3865e+04, 2.7000e+01, 3.2600e+02, 0.0000e+00, 0.0000e+00],
          [1.8330e+03, 0.0000e+00, 2.0000e+00, 0.0000e+00, 0.0000e+00],
          [1.9580e+03, 0.0000e+00, 1.2620e+03, 0.0000e+00, 0.0000e+00],
          [3.5100e+02, 0.0000e+00, 3.7000e+01, 0.0000e+00, 0.0000e+00],
          [4.0000e+00, 0.0000e+00, 3.0000e+00, 0.0000e+00, 0.0000e+00]]),
   'confMat_std': array([[0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.]]),
   'f1': 0.8805818742793907,
   'precision': 0.8635857289880641,
   'recall': 0.9085729242167996},
  'class': {'acc': array([0.99201683, 0.        , 0.39192547, 0.        , 0.        ]),
   'fsc': array([0.95121925, 0.        , 0.52041237, 0.        , 0.        ]),
   'hm': array([0.3778355 , 0.        , 0.56181434, 0.        , 0.        ]),
   'ppv': array([0.91364

In [74]:
export_results(results, 'validation')

/Users/leandrobmarinho/Documents/ECG-Arritmia-Paper1/Results_Navar/SCM/Experiment_02_scikitLearn__allCLF_onDS2_allFeatures
MLP-96
SVM-RBF
Naive-Bayes


/Users/leandrobmarinho/Documents/ECG-Arritmia-Paper1/Results_Navar/SCM/utils/experiment.py:293: RuntimeWarning: invalid value encountered in true_divide
  return np.around((confMat / np.sum(confMat,axis=1)[:,None])*100,2)


# Computing train time

In [ ]:
path = os.getcwd()
rootDir, _ = path.split('ECG-Arritmia-Paper1')
path = 'ECG-Arritmia-Paper1/Database_extracted/Dataset_with_features'
filename = 'MIT-BIH__DS2_5classes__SCM.csv'
dataset = pd.read_csv(os.path.join(rootDir, path, filename));

In [19]:
from sklearn.externals import joblib
from sklearn.naive_bayes import GaussianNB
from time import time

bayes_clf = GaussianNB()
bayes_clf.fit(data_std, data_label)

random_search = joblib.load('./models/classifiers.pkl')
svm_clf = random_search['SVM-RBF'].best_estimator_
mlp_clf = random_search['MLP'].best_estimator_

classifiers = {'SVM-RBF': svm_clf} #'MLP': mlp_clf, 

for key,value  in classifiers.items():
    start_time = time()
    value.fit(data_std, data_label)
    end_time = time() - start_time
    print('Elapsed time {}: {}'.format(key, end_time))

Elapsed time SVM-RBF: 88.95117211341858


In [ ]:
random_search = joblib.load('./models/classifiers.pkl')
print(random_search)